<a href="https://colab.research.google.com/github/DimbikeY/Neuronetworks/blob/main/Keras%20Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Применение Keras Tuner для автоматического подбора гиперпараметров
К гиперпараметрам относят: 
- Количество слоёв нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях 
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [1]:
!git clone https://github.com/keras-team/keras-tuner.git

Cloning into 'keras-tuner'...
remote: Enumerating objects: 8469, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 8469 (delta 156), reused 172 (delta 94), pack-reused 8216
Receiving objects: 100% (8469/8469), 2.00 MiB | 11.61 MiB/s, done.
Resolving deltas: 100% (5972/5972), done.


In [ ]:
%cd keras-tuner
!pip install .

In [5]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from google.colab import files
from kerastuner import RandomSearch, Hyperband, BayesianOptimization
import numpy as np
import matplotlib as plt
from PIL import Image
%matplotlib inline

In [7]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255
x_test = x_test / 255
y_train = utils.to_categorical(y_train, 10)
y_tet = utils.to_categorical(y_test, 10)

## Задаём функцию создания нейронной сети

In [15]:
def build_model(hp): # hp - движок гиперпараметра keras tuner
  model = Sequential()
  activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
  for i in range(hp.Range('num_layers', 2, 5)):
    model.add(Dense(units = hp.Int('units_input', min_value = 128, # полносвязный слой с разным количество нейронов
                                   max_value = 1024,               # минимальное количество нейронов 128, макс - 1024
                                   step = 32),
                  input_dim = 784,
                  activation=activation_choice))
  model.add(Dense(10, activation = 'softmax'))
  model.compile(
      optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop, SGD']),
      loss = 'categorical_crossentropy',
      metrics = ['accuracy'])
  return model

## Создаём tuner

In [19]:
# RandomSearch - случайный поиск
# Hyperband - многорукий бандит
# BayesianOptimization - байесовская оптимизация

tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=10,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запуск подбора гиперпараметров

Пространство поиска

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}


In [21]:
tuner.search(x_train, y_train, batch_size = 256, epochs = 3, validation_split = 0.2,
             verbose = 1)

Trial 12 Complete [00h 00m 21s]
val_accuracy: 0.7820833325386047

Best val_accuracy So Far: 0.7899166941642761
Total elapsed time: 00h 00m 21s


In [22]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units_input: 832
Score: 0.7899166941642761
Trial summary
Hyperparameters:
units_input: 768
Score: 0.7866666913032532
Trial summary
Hyperparameters:
units_input: 992
Score: 0.7854166626930237
Trial summary
Hyperparameters:
units_input: 864
Score: 0.784583330154419
Trial summary
Hyperparameters:
units_input: 512
Score: 0.7831666469573975
Trial summary
Hyperparameters:
units_input: 1024
Score: 0.7820833325386047
Trial summary
Hyperparameters:
units_input: 608
Score: 0.7793333530426025
Trial summary
Hyperparameters:
units_input: 576
Score: 0.7790833115577698
Trial summary
Hyperparameters:
units_input: 256
Score: 0.7751666903495789
Trial summary
Hyperparameters:
units_input: 320
Score: 0.7711666822433472


Получим 3 лучших модели

In [26]:
models = tuner.get_best_models(num_models=3)

In [27]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 832)               653120    
                                                                 
 dense_1 (Dense)             (None, 10)                8330      
                                                                 
Total params: 661,450
Trainable params: 661,450
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored